# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [30]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [31]:
csv_file = os.path.join('../output_data/cities.csv')

city_df = pd.read_csv(csv_file)
city_df


,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
0,new norfolk,AU,64.00,32,29,10.00,-42.7826,147.0587,1615686654
1,mataura,NZ,75.00,43,40,15.01,-46.1927,168.8643,1615686640
2,carnarvon,AU,89.60,48,32,16.11,-24.8667,113.6333,1615686702
3,vaini,TO,80.60,78,40,8.05,-21.2000,-175.2000,1615686650
4,alofi,NU,86.00,66,98,8.05,-19.0595,-169.9187,1615686821
...,...,...,...,...,...,...,...,...,...
557,shitkino,RU,6.39,89,100,16.28,56.3703,98.3543,1615686955
558,port hedland,AU,84.20,65,40,24.16,-20.3167,118.5667,1615686696
559,porto novo,BJ,84.20,79,40,11.50,6.4965,2.6036,1615686955
560,mumford,GH,81.45,79,100,12.80,5.2618,-0.7590,1615686956


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [32]:
gmaps.configure(api_key=g_key)

In [33]:
locations = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"]

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
heat_layer.max_intensity = 100
heat_layer.point_radius = 5


fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [34]:
ideal_df = city_df[(city_df['Temperature'] >= 75) & (city_df['Temperature'] <= 90)]
ideal_df = ideal_df[ideal_df['Windspeed'] <= 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] <= 10]
ideal_df = ideal_df[ideal_df['Humidity'] <= 70]
hotel_df = ideal_df
hotel_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date
16,kilindoni,TZ,80.47,66,0,2.89,-7.9139,39.6668,1615686823
65,bantogon,PH,83.32,67,0,6.11,6.5333,124.0333,1615686835
66,atuona,PF,79.57,62,0,5.66,-9.8000,-139.0333,1615686639
107,tubuala,PA,80.60,65,0,8.05,9.5167,-79.0333,1615686845
117,avarua,CK,84.20,70,3,4.61,-21.2078,-159.7750,1615686644
121,san pedro de ycuamandiyu,PY,78.10,44,8,3.69,-24.0917,-57.0764,1615686849
127,mungeli,IN,75.20,20,0,2.71,22.0667,81.6833,1615686850
164,namatanai,PG,85.48,57,0,4.05,-3.6667,152.4333,1615686771
225,akyab,MM,78.82,48,0,5.32,20.1500,92.9000,1615686873
259,khetia,IN,75.96,19,0,2.26,21.6667,74.5833,1615686881


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date,Hotel Name
16,kilindoni,TZ,80.47,66,0,2.89,-7.9139,39.6668,1615686823,
65,bantogon,PH,83.32,67,0,6.11,6.5333,124.0333,1615686835,
66,atuona,PF,79.57,62,0,5.66,-9.8000,-139.0333,1615686639,
107,tubuala,PA,80.60,65,0,8.05,9.5167,-79.0333,1615686845,
117,avarua,CK,84.20,70,3,4.61,-21.2078,-159.7750,1615686644,
121,san pedro de ycuamandiyu,PY,78.10,44,8,3.69,-24.0917,-57.0764,1615686849,
127,mungeli,IN,75.20,20,0,2.71,22.0667,81.6833,1615686850,
164,namatanai,PG,85.48,57,0,4.05,-3.6667,152.4333,1615686771,
225,akyab,MM,78.82,48,0,5.32,20.1500,92.9000,1615686873,
259,khetia,IN,75.96,19,0,2.26,21.6667,74.5833,1615686881,


In [36]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Latitude']
        lng = row['Longitude']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Country,Temperature,Humidity,Cloudiness,Windspeed,Latitude,Longitude,Date,Hotel Name
16,kilindoni,TZ,80.47,66,0,2.89,-7.9139,39.6668,1615686823,Juani Beach Bungalows
65,bantogon,PH,83.32,67,0,6.11,6.5333,124.0333,1615686835,Kalamansig Greenhome
66,atuona,PF,79.57,62,0,5.66,-9.8000,-139.0333,1615686639,Hôtel Hiva Oa Hanakee Lodge
107,tubuala,PA,80.60,65,0,8.05,9.5167,-79.0333,1615686845,NaN
117,avarua,CK,84.20,70,3,4.61,-21.2078,-159.7750,1615686644,Coast Cook Islands - 2 bedroom Beachfront Luxu...
121,san pedro de ycuamandiyu,PY,78.10,44,8,3.69,-24.0917,-57.0764,1615686849,Hotel San Caye
127,mungeli,IN,75.20,20,0,2.71,22.0667,81.6833,1615686850,Hotel Puneet
164,namatanai,PG,85.48,57,0,4.05,-3.6667,152.4333,1615686771,Namatanai Hotel
225,akyab,MM,78.82,48,0,5.32,20.1500,92.9000,1615686873,Sittwe Hotel
259,khetia,IN,75.96,19,0,2.26,21.6667,74.5833,1615686881,NaN


In [37]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [38]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))